In [ ]:
# Set environment variables FIRST, before any LangChain imports
import os
os.environ["LANGSMITH_TRACING"] = "false"
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["GOOGLE_API_KEY"] = ""

# Now import LangChain modules
from langchain.chat_models import init_chat_model
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_community.document_loaders import PyPDFLoader
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings


In [47]:


LANGSMITH_API_KEY = os.environ["LANGSMITH_API_KEY"]
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
LANGSMITH_TRACING = os.environ["LANGSMITH_TRACING"]

### LLM model

In [48]:
from langchain.chat_models import init_chat_model

llm = ChatOllama(model="llama3.2:3b")

In [49]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Azerbaijani. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]


In [50]:
llm_google = init_chat_model(
    "gemini-2.0-flash-exp", 
    model_provider="google_genai",
    google_api_key=GOOGLE_API_KEY
)

### Embedding Model

In [76]:
embeddings_ollama = OllamaEmbeddings(model="nomic-embed-text", base_url="http://localhost:11434")

In [66]:
from langchain_huggingface import HuggingFaceEmbeddings

# The rest of your code
embedding_hg = HuggingFaceEmbeddings(
     model_name="LocalDoc/az-en-MiniLM-L6-v2")

c:\Users\NCAFE\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [92]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embedding_hg)

In [94]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
from langchain_community.document_loaders import PyPDFLoader

# file_path = "execution-of-sentences.pdf"
file_path = "emek-mecellesi-1-20.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# we can later optimize the prompt, for now lets use predefined prompt from hub
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm_google.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

c:\Users\NCAFE\AppData\Local\Programs\Python\Python313\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [90]:
question = "Which institutions and bodies are responsible for executing different types of punishments?"

response = graph.invoke({"question": question})
import textwrap
wrapped_text = textwrap.fill(response["answer"], width=100)
print(wrapped_text)

The bailiff of the court executes community service punishments. Fines are executed by the
legislation upon the presentation of the institution or body executing the main punishment. The
executive officer at the place of residence of the convict executes the punishment in the form of
deprivation of the right to hold a certain position or engage in certain activity.
